In [1]:
library(tidyverse)

# Custom package
library(rutils)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [3]:
dset_idx <- 3

# Load and filter survival data

In [4]:
event_code <- list("Alive" = 0, "Dead" = 1)
covariate_cols_no_figo <- c("age_at_diagnosis", "race", "ethnicity")
covariate_cols <- c("figo_stage", covariate_cols_no_figo)
dep_cols <- c("vital_status", "survival_time")

In [5]:
survival_path <- paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/survival_data.tsv")
survival_df <- load_survival_df(survival_path, event_code)

Parsed with column specification:
cols(
  sample_name = col_character(),
  vital_status = col_character(),
  survival_time = col_double(),
  figo_stage = col_character(),
  days_to_last_follow_up = col_double(),
  days_to_death = col_double(),
  age_at_diagnosis = col_double(),
  age_at_index = col_double(),
  height = col_double(),
  weight = col_double(),
  bmi = col_double(),
  race = col_character(),
  ethnicity = col_character()
)


In [6]:
filtered_survival_df <- survival_df %>%
    decode_figo_stage(to = "c") %>%
    dplyr::select(sample_name, figo_stage, race, ethnicity, age_at_diagnosis) %>% # make sure using same samples as classification models
    dplyr::filter(rowSums(is.na(.)) == 0) %>%
    dplyr::select(sample_name, figo_stage)

# Load normalized matrisome count data

In [7]:
norm_matrisome_counts_path <- paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/norm_matrisome_counts.tsv")
norm_survival_counts_t_df <- read_tsv(norm_matrisome_counts_path) %>%
    dplyr::select(c("geneID", filtered_survival_df$sample_name)) %>%
    transpose_df("geneID", "sample_name")

Parsed with column specification:
cols(
  .default = col_double(),
  geneID = col_character()
)
See spec(...) for full column specifications.


In [8]:
joined_df <- filtered_survival_df %>%
    inner_join(norm_survival_counts_t_df, by = "sample_name")

In [9]:
joined_df %>%
    group_by(figo_stage) %>%
    summarize(n = n())

`summarise()` ungrouping output (override with `.groups` argument)


figo_stage,n
<chr>,<int>
figo_stage_1,73
figo_stage_2,19
figo_stage_3,38
figo_stage_4,7


In [10]:
# Some genes contain the '-' symbol, which affects formulae
colnames(joined_df) <- gsub("-", "_", colnames(joined_df))

In [11]:
gene_names <- colnames(joined_df[-c(1:2)])

In [17]:
waov_df <- colwise_anova(joined_df, "figo_stage", gene_names, "geneID", adjust_method = "BH")

In [18]:
# write_tsv(waov_df, paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_welch_anova_results.tsv"))

In [19]:
waov_df %>%
    dplyr::filter(padj < 0.05)

geneID,pval,padj
<chr>,<dbl>,<dbl>
MUC3A,5.083115e-04,4.498752e-02
IL7,2.304654e-05,1.106965e-02
SERPINA3,5.864586e-04,4.498752e-02
TGM3,6.041712e-04,4.498752e-02
FGF14,5.232242e-04,4.498752e-02
LGI2,3.430677e-05,1.106965e-02
GDF7,1.368575e-04,2.650798e-02
IHH,1.516764e-08,1.468228e-05
MEGF9,4.229478e-04,4.498752e-02


In [22]:
waov_df$geneID[str_detect(waov_df$geneID, "_")]
waov_df$geneID <- gsub("_", "-", waov_df$geneID)
waov_df$geneID[str_detect(waov_df$geneID, "_")]

[1] "INS_IGF2"

character(0)